In [1]:
# libraries and settings
import gc
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy.stats import pearsonr
from sklearn.linear_model import Ridge
import lightgbm as lgb
import optuna
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_parquet('../data/input/train.parquet')
df_test = pd.read_parquet('../data/input/test.parquet')

In [3]:
# Clustering the features
X_cols = df_train.columns[df_train.columns.str.startswith('X')]
clusters = pd.Series(pd.read_csv('../data/intermediate/clusters.csv', index_col = 0).iloc[:,0])

# shuffle the features
clusters = clusters.sample(frac = 1, random_state = 42)

# sampling teh clusters
clusters_dict = defaultdict(list)

for c in np.unique(clusters):
    X = clusters[clusters == c]
    clusters_dict[c] = X.index.tolist()

total = 0
features_list = []
n_features = len(clusters)

while total < n_features:
    features = []
    for c in clusters_dict.keys():
        if len(clusters_dict[c]) > 0:
            features.append(clusters_dict[c][-1])
            clusters_dict[c].pop()
            total += 1
    features_list.append(features)
    features = list([])
    
while len(features_list[-1]) < 10:
    features_list.pop()

for i in range(len(features_list)):
    features_list[i] = ['bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume'] + features_list[i]

In [5]:
""" **************** ENSEMBLE MODEL ****************** """
class LightGBMTimeSeriesEnsemble:
    def __init__(self, n_models, n_trials, feature_sets, alphas=None,
                 metric="l2", random_seed=42):
        """
        n_models  : number of base learners (e.g. 50)
        n_trials  : Optuna trials per learner
        alphas    : candidate ridge regularization strengths
        metric    : LightGBM eval metric (for early stopping)
        """
        self.n_models = n_models
        self.n_trials = n_trials
        self.feature_sets = feature_sets
        self.metric = metric
        self.alphas = alphas if alphas is not None else np.logspace(-2, 5, 30)
        self.seed = random_seed

        self.best_params_list = []
        self.best_iterations = []
        self.models = []
        self.ridge = None
        self.best_alpha = None

    def _optuna_objective(self, trial, X_tr, y_tr, X_val, y_val):
        params = {
            'objective': 'regression',
            'metric': 'rmse',  
            'verbosity': -1,
            'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
            'num_leaves': trial.suggest_int('num_leaves', 16, 64),
            'max_depth': trial.suggest_int('max_depth', 2, 6),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 0.5),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.2, 0.6),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
            'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
            'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
            'num_boost_round': trial.suggest_int('num_boost_round', 200,500),
            'early_stopping_rounds': 50
        }
        model = lgb.train(
            params,
            lgb.Dataset(X_tr, y_tr),
            valid_sets=[lgb.Dataset(X_val, y_val)]
        )
        pred = model.predict(X_val)
        corr, _ = pearsonr(pred, y_val)
        return -corr

    def fit(self, X_train, y_train, X_val, y_val):
        """
        1) Fit each base model on X_train[i::n] versus y_train[i::n] with Optuna tuning.
        Next steps are carried out in the fit_ridge function:
        2) Make predictions of each base model on *full* X_train => (n_samples_train x n_models)
        3) Ridge-fit on (train_preds, y_train)
        4) Select alpha by evaluating ridge on full X_val predictions
        """
        self.models = []
        self.best_params_list = []
        self.best_iterations = []

        for j in range(len(self.feature_sets)):
            for i in range(self.n_models):
                print(f'Training on feature set {j+1}/{len(self.feature_sets)}, model {i+1}/{self.n_models}')
                
                Xt = X_train.iloc[i::self.n_models][self.feature_sets[j]]
                yt = y_train.iloc[i::self.n_models]
                Xv = X_val.iloc[i::self.n_models][self.feature_sets[j]]
                yv = y_val.iloc[i::self.n_models]
    
                # Tune
                sampler = optuna.samplers.TPESampler(seed=self.seed)
                study = optuna.create_study(direction='minimize', sampler=sampler)
                study.optimize(
                    lambda trial: self._optuna_objective(trial, Xt, yt, Xv, yv),
                    n_trials=self.n_trials,
                )
                best_params = study.best_params
                best_params['seed'] = self.seed
                
    
                # Train model with best params
                model = lgb.train(
                    {
                    **best_params, 
                    'objective':'regression',
                    'metric':self.metric,
                    'verbosity':-1, 
                    'early_stopping_rounds': 50
                    },
                    lgb.Dataset(Xt, yt),
                    valid_sets=[lgb.Dataset(Xv, yv)]
                )
                best_params['num_boost_round'] = model.best_iteration
                self.best_params_list.append(best_params)
                self.best_iterations.append(model.best_iteration)
                self.models.append(model)

        self.fit_ridge(X_train, y_train, X_val, y_val, self.alphas)
        
        
    def fit_ridge(self, X_train, y_train, X_val, y_val, alphas):
        """
        1) Make predictions of each base model on *full* X_train => (n_samples_train x n_models)
        2) Ridge-fit on (train_preds, y_train)
        3) Select alpha by evaluating ridge on full X_val predictions
        """
        train_preds = []
        val_preds   = []
        idx = 0

         # --- predictions on full training and validation set ---
        for subset in self.feature_sets:
            for _ in range(self.n_models):
                m  = self.models[idx]
                it = self.best_iterations[idx]
                train_preds.append(m.predict(X_train[subset], num_iteration=it))
                val_preds.append(m.predict(X_val[subset],   num_iteration=it))
                idx += 1
        
        train_stack = np.column_stack(train_preds)
        val_stack   = np.column_stack(val_preds)
       
        self.alphas = alphas
        best_corr = -np.inf
        for a in self.alphas:
            r = Ridge(alpha=a)
            r.fit(train_stack, y_train)
            corr, _ = pearsonr(r.predict(val_stack), y_val)
            print(f' ridge alpha: {a:.5f}, val corr: {corr:.5f}')
            if corr > best_corr:
                best_corr, self.best_alpha, self.ridge = corr, a, r

        print(f'Best ridge alpha: {self.best_alpha:.5f}, val corr: {best_corr:.5f}')
    
    
    def refit_full(self, X_full, y_full):
        """
        Retrain base models and the ridge regression model on full data using best params and alpha.
        """
        self.models = []
        idx = 0
        for subset in self.feature_sets:
            for _ in range(self.n_models):
                print(f'Refitting model {idx+1}/{self.n_models*len(self.feature_sets)} on full data...')
                params  = self.best_params_list[idx]
                model = lgb.train(
                    {
                    **params,  
                    'objective':'regression',
                    'metric':self.metric,
                    'verbosity':-1
                    },
                    lgb.Dataset(X_full[subset], y_full)
                )
                self.models.append(model)
                idx += 1
        
        preds = []
        idx = 0
         # --- predictions on full dataset
        for subset in self.feature_sets:
            for _ in range(self.n_models):
                m  = self.models[idx]
                it = self.best_iterations[idx]
                preds.append(m.predict(X_full[subset], num_iteration=it))
                idx += 1
                
        full_stack = stack = np.column_stack(preds)       
        self.ridge = Ridge(alpha=self.best_alpha)
        self.ridge.fit(full_stack, y_full)

    def predict(self, X):
        """
        Predict stacked output on full dataset X
        """
        preds = []
        idx = 0
        for subset in self.feature_sets:
            for _ in range(self.n_models):
                m  = self.models[idx]
                it = self.best_iterations[idx]
                preds.append(m.predict(X[subset], num_iteration=it))
                idx += 1
        stack = np.column_stack(preds)
        
        return self.ridge.predict(stack)



In [ ]:
df_train.sort_index(inplace = True)
n_train = round(df_train.shape[0]*0.8)
X_train = df_train.iloc[:n_train].drop(columns = ['label'])
X_val = df_train.iloc[n_train:].drop(columns = ['label'])
y_train = df_train.iloc[:n_train]['label'] 
y_val = df_train.iloc[n_train:]['label'] 

ensemble = LightGBMTimeSeriesEnsemble(n_models=5, n_trials=50, feature_sets=features_list)
ensemble.fit(X_train, y_train, X_val, y_val)

[I 2025-08-19 14:47:32,047] A new study created in memory with name: no-name-94a59bad-11a4-4401-87c3-51df3e531bc1


Training on feature set 1/12, model 1/5


[I 2025-08-19 14:47:33,925] Trial 0 finished with value: -0.08080226745370152 and parameters: {'learning_rate': 0.015355286838886862, 'num_leaves': 62, 'max_depth': 5, 'min_data_in_leaf': 64, 'feature_fraction': 0.1624074561769746, 'bagging_fraction': 0.26239780813448105, 'bagging_freq': 1, 'lambda_l1': 4.330880728874676, 'lambda_l2': 3.005575058716044, 'num_boost_round': 413}. Best is trial 0 with value: -0.08080226745370152.
[I 2025-08-19 14:47:35,966] Trial 1 finished with value: -0.0829087427784682 and parameters: {'learning_rate': 0.005318033256270142, 'num_leaves': 63, 'max_depth': 6, 'min_data_in_leaf': 29, 'feature_fraction': 0.17272998688284025, 'bagging_fraction': 0.27336180394137355, 'bagging_freq': 4, 'lambda_l1': 2.6237821581611893, 'lambda_l2': 2.1597250932105787, 'num_boost_round': 287}. Best is trial 1 with value: -0.0829087427784682.
[I 2025-08-19 14:47:37,512] Trial 2 finished with value: -0.08354512870665791 and parameters: {'learning_rate': 0.03126143958203108, 'num

Training on feature set 1/12, model 2/5


[I 2025-08-19 14:49:10,237] A new study created in memory with name: no-name-2a66fce1-b68f-4873-af17-66419fd7296d
[I 2025-08-19 14:49:11,710] Trial 0 finished with value: -0.07518809983599434 and parameters: {'learning_rate': 0.015355286838886862, 'num_leaves': 62, 'max_depth': 5, 'min_data_in_leaf': 64, 'feature_fraction': 0.1624074561769746, 'bagging_fraction': 0.26239780813448105, 'bagging_freq': 1, 'lambda_l1': 4.330880728874676, 'lambda_l2': 3.005575058716044, 'num_boost_round': 413}. Best is trial 0 with value: -0.07518809983599434.
[I 2025-08-19 14:49:14,105] Trial 1 finished with value: -0.09776442774770872 and parameters: {'learning_rate': 0.005318033256270142, 'num_leaves': 63, 'max_depth': 6, 'min_data_in_leaf': 29, 'feature_fraction': 0.17272998688284025, 'bagging_fraction': 0.27336180394137355, 'bagging_freq': 4, 'lambda_l1': 2.6237821581611893, 'lambda_l2': 2.1597250932105787, 'num_boost_round': 287}. Best is trial 1 with value: -0.09776442774770872.
[I 2025-08-19 14:49:1

...

[I 2025-08-19 14:58:51,897] Trial 0 finished with value: -0.024168096612199604 and parameters: {'learning_rate': 0.015355286838886862, 'num_leaves': 62, 'max_depth': 5, 'min_data_in_leaf': 64, 'feature_fraction': 0.1624074561769746, 'bagging_fraction': 0.26239780813448105, 'bagging_freq': 1, 'lambda_l1': 4.330880728874676, 'lambda_l2': 3.005575058716044, 'num_boost_round': 413}. Best is trial 0 with value: -0.024168096612199604.
[I 2025-08-19 14:58:53,155] Trial 1 finished with value: -0.037934214869067125 and parameters: {'learning_rate': 0.005318033256270142, 'num_leaves': 63, 'max_depth': 6, 'min_data_in_leaf': 29, 'feature_fraction': 0.17272998688284025, 'bagging_fraction': 0.27336180394137355, 'bagging_freq': 4, 'lambda_l1': 2.6237821581611893, 'lambda_l2': 2.1597250932105787, 'num_boost_round': 287}. Best is trial 1 with value: -0.037934214869067125.
[I 2025-08-19 14:58:54,152] Trial 2 finished with value: -0.04815460330344358 and parameters: {'learning_rate': 0.03126143958203108

Training on feature set 3/12, model 1/5


[I 2025-08-19 14:59:47,108] Trial 0 finished with value: -0.04445493508201475 and parameters: {'learning_rate': 0.015355286838886862, 'num_leaves': 62, 'max_depth': 5, 'min_data_in_leaf': 64, 'feature_fraction': 0.1624074561769746, 'bagging_fraction': 0.26239780813448105, 'bagging_freq': 1, 'lambda_l1': 4.330880728874676, 'lambda_l2': 3.005575058716044, 'num_boost_round': 413}. Best is trial 0 with value: -0.04445493508201475.
[I 2025-08-19 14:59:48,009] Trial 1 finished with value: -0.06939055786423222 and parameters: {'learning_rate': 0.005318033256270142, 'num_leaves': 63, 'max_depth': 6, 'min_data_in_leaf': 29, 'feature_fraction': 0.17272998688284025, 'bagging_fraction': 0.27336180394137355, 'bagging_freq': 4, 'lambda_l1': 2.6237821581611893, 'lambda_l2': 2.1597250932105787, 'num_boost_round': 287}. Best is trial 1 with value: -0.06939055786423222.
[I 2025-08-19 14:59:48,777] Trial 2 finished with value: -0.053960029145192316 and parameters: {'learning_rate': 0.03126143958203108, '

Training on feature set 3/12, model 2/5


[I 2025-08-19 15:00:55,977] Trial 0 finished with value: -0.06218539508214488 and parameters: {'learning_rate': 0.015355286838886862, 'num_leaves': 62, 'max_depth': 5, 'min_data_in_leaf': 64, 'feature_fraction': 0.1624074561769746, 'bagging_fraction': 0.26239780813448105, 'bagging_freq': 1, 'lambda_l1': 4.330880728874676, 'lambda_l2': 3.005575058716044, 'num_boost_round': 413}. Best is trial 0 with value: -0.06218539508214488.
[I 2025-08-19 15:00:57,324] Trial 1 finished with value: -0.06512459586665434 and parameters: {'learning_rate': 0.005318033256270142, 'num_leaves': 63, 'max_depth': 6, 'min_data_in_leaf': 29, 'feature_fraction': 0.17272998688284025, 'bagging_fraction': 0.27336180394137355, 'bagging_freq': 4, 'lambda_l1': 2.6237821581611893, 'lambda_l2': 2.1597250932105787, 'num_boost_round': 287}. Best is trial 1 with value: -0.06512459586665434.
[I 2025-08-19 15:00:58,567] Trial 2 finished with value: -0.0783350357366696 and parameters: {'learning_rate': 0.03126143958203108, 'nu

In [ ]:
# further fine-tuning the ridge regression regularization parameter
ensemble.fit_ridge(X_train, y_train, X_val, y_val, range(2000, 15000, 5000))

In [12]:
ensemble.refit_full(df_train.drop(columns='label'), df_train['label'])

y_test_pred = ensemble.predict(df_test)
y_test_pred_pd = pd.Series(y_test_pred, name = 'prediction')
y_test_pred_pd.index = df_test.index
y_test_pred_pd.to_csv('../data/output/submission6.csv')

Refitting model 1/60 on full data...
Refitting model 2/60 on full data...
Refitting model 3/60 on full data...
Refitting model 4/60 on full data...
Refitting model 5/60 on full data...
Refitting model 6/60 on full data...
Refitting model 7/60 on full data...
Refitting model 8/60 on full data...
Refitting model 9/60 on full data...
Refitting model 10/60 on full data...
Refitting model 11/60 on full data...
Refitting model 12/60 on full data...
Refitting model 13/60 on full data...
Refitting model 14/60 on full data...
Refitting model 15/60 on full data...
Refitting model 16/60 on full data...
Refitting model 17/60 on full data...
Refitting model 18/60 on full data...
Refitting model 19/60 on full data...
Refitting model 20/60 on full data...
Refitting model 21/60 on full data...
Refitting model 22/60 on full data...
Refitting model 23/60 on full data...
Refitting model 24/60 on full data...
Refitting model 25/60 on full data...
Refitting model 26/60 on full data...
Refitting model 27/60